**Copyright 2021 The TensorFlow Authors.**

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Exemplo do Keras de clustering que preserva a esparsidade

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/combine/sparse_clustering_example"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/model_optimization/guide/combine/sparse_clustering_example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/model_optimization/guide/combine/sparse_clustering_example.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/model_optimization/guide/combine/sparse_clustering_example.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

Este é um exemplo completo que mostra o uso da API de **clustering que preserva a esparsidade**, parte do pipeline de otimização colaborativa do Kit de ferramentas para otimização de modelos do TensorFlow.

### Outras páginas

Para uma introdução sobre o pipeline e outras técnicas disponíveis, confira a [página de visão geral da otimização colaborativa](https://www.tensorflow.org/model_optimization/guide/combine/collaborative_optimization).

### Conteúdo

Neste tutorial, você:

1. Treinará um modelo `tf.keras` para o dataset MNIST do zero.
2. Ajustará o modelo com a esparsidade, verá a exatidão e observará se o pruning teve êxito.
3. Aplicará o clustering de peso ao modelo após o pruning e observará a perda de esparsidade.
4. Aplicará o clustering que preserva a esparsidade no modelo após o pruning e observará se a esparsidade aplicada antes foi preservada.
5. Gerará um modelo do TFLite e verificará se a exatidão foi preservada no modelo agrupado e podado.
6. Comparará os tamanhos de modelos diferentes para observar os benefícios da compressão ao aplicar a esparsidade, seguida pela técnica de clustering que preserva a esparsidade para otimização colaborativa.

## Configuração

Você pode executar este Notebook do Jupyter no seu [virtualenv](https://www.tensorflow.org/install/pip?lang=python3#2.-create-a-virtual-environment-recommended) local ou no [colab](https://colab.sandbox.google.com/). Para mais detalhes sobre como configurar as dependências, consulte o [guia de instalação](https://www.tensorflow.org/model_optimization/guide/install). 

In [ ]:
! pip install -q tensorflow-model-optimization

In [ ]:
import tensorflow as tf

import numpy as np
import tempfile
import zipfile
import os

## Treine um modelo tf.keras para o MNIST que passará pelo pruning e clustering

In [ ]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images  = test_images / 255.0

model = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=(28, 28)),
  tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3),
                         activation=tf.nn.relu),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    validation_split=0.1,
    epochs=10
)

### Avalie o modelo de referência e salve-o para usar mais tarde

In [ ]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

## Faça o pruning e ajuste o modelo para 50% da esparsidade

Aplique a API `prune_low_magnitude()` para podar todo o modelo pré-treinado a fim de obter o modelo que passará pelo clustering na próxima etapa. Para saber como melhor usar a API para alcançar a melhor proporção de compressão e ainda manter a exatidão alvo, consulte o [guia completo de pruning](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide).

### Defina o modelo e aplique a API de esparsidade

Observe que é usado o modelo pré-treinado.

In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(0.5, begin_step=0, frequency=100)
  }

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep()
]

pruned_model = prune_low_magnitude(model, **pruning_params)

# Use smaller learning rate for fine-tuning
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

pruned_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

pruned_model.summary()

### Ajuste o modelo, verifique a esparsidade e compare a exatidão dele com a referência

Ajuste o modelo com o pruning para 3 épocas.

In [ ]:
# Fine-tune model
pruned_model.fit(
  train_images,
  train_labels,
  epochs=3,
  validation_split=0.1,
  callbacks=callbacks)

Defina funções helper para calcular e imprimir a esparsidade do modelo.

In [ ]:
def print_model_weights_sparsity(model):

    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Wrapper):
            weights = layer.trainable_weights
        else:
            weights = layer.weights
        for weight in weights:
            if "kernel" not in weight.name or "centroid" in weight.name:
                continue
            weight_size = weight.numpy().size
            zero_num = np.count_nonzero(weight == 0)
            print(
                f"{weight.name}: {zero_num/weight_size:.2%} sparsity ",
                f"({zero_num}/{weight_size})",
            )

Verifique se os kernels do modelo foram podados corretamente. Precisamos retirar o wrapper de pruning primeiro. Também vamos criar uma cópia profunda do modelo que será usado na próxima etapa.

In [ ]:
stripped_pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

print_model_weights_sparsity(stripped_pruned_model)

stripped_pruned_model_copy = tf.keras.models.clone_model(stripped_pruned_model)
stripped_pruned_model_copy.set_weights(stripped_pruned_model.get_weights())

## Aplique o clustering e o clustering que preserva a esparsidade e confira o efeito na esparsidade do modelo em ambos os casos

Em seguida, vamos aplicar o clustering e o clustering que preserva a esparsidade no modelo podado e observar se o último preserva a esparsidade no modelo podado. Observe que retiramos os wrappers de pruning do modelo podado com `tfmot.sparsity.keras.strip_pruning` antes de aplicar a API de clustering.

In [ ]:
# Clustering
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 8,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
}

clustered_model = cluster_weights(stripped_pruned_model, **clustering_params)

clustered_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

print('Train clustering model:')
clustered_model.fit(train_images, train_labels,epochs=3, validation_split=0.1)


stripped_pruned_model.save("stripped_pruned_model_clustered.h5")

# Sparsity preserving clustering
from tensorflow_model_optimization.python.core.clustering.keras.experimental import (
    cluster,
)

cluster_weights = cluster.cluster_weights

clustering_params = {
  'number_of_clusters': 8,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS,
  'preserve_sparsity': True
}

sparsity_clustered_model = cluster_weights(stripped_pruned_model_copy, **clustering_params)

sparsity_clustered_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

print('Train sparsity preserving clustering model:')
sparsity_clustered_model.fit(train_images, train_labels,epochs=3, validation_split=0.1)

Verifique a esparsidade de ambos os modelos.

In [ ]:
print("Clustered Model sparsity:\n")
print_model_weights_sparsity(clustered_model)
print("\nSparsity preserved clustered Model sparsity:\n")
print_model_weights_sparsity(sparsity_clustered_model)

## Crie modelos 1,6x menores com o clustering

Defina a função helper para obter um arquivo de modelo compactado.

In [ ]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in kilobytes.

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)/1000

In [ ]:
# Clustered model
clustered_model_file = 'clustered_model.h5'

# Save the model.
clustered_model.save(clustered_model_file)
    
#Sparsity Preserve Clustered model
sparsity_clustered_model_file = 'sparsity_clustered_model.h5'

# Save the model.
sparsity_clustered_model.save(sparsity_clustered_model_file)
    
print("Clustered Model size: ", get_gzipped_model_size(clustered_model_file), ' KB')
print("Sparsity preserved clustered Model size: ", get_gzipped_model_size(sparsity_clustered_model_file), ' KB')

## Crie um modelo do TFLite ao combinar o clustering de peso que preserva a esparsidade e a quantização pós-treinamento

Retire os wrappers de clustering e converta para o TFLite.

In [ ]:
stripped_sparsity_clustered_model = tfmot.clustering.keras.strip_clustering(sparsity_clustered_model)

converter = tf.lite.TFLiteConverter.from_keras_model(stripped_sparsity_clustered_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
sparsity_clustered_quant_model = converter.convert()

_, pruned_and_clustered_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_and_clustered_tflite_file, 'wb') as f:
  f.write(sparsity_clustered_quant_model)

print("Sparsity preserved clustered Model size: ", get_gzipped_model_size(sparsity_clustered_model_file), ' KB')
print("Sparsity preserved clustered and quantized TFLite model size:",
       get_gzipped_model_size(pruned_and_clustered_tflite_file), ' KB')

## Veja a persistência da exatidão do TF para o TFLite

In [ ]:
def eval_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print(f"Evaluated on {i} results so far.")
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

Avalie o modelo após o pruning, o clustering e a quantização e veja se a exatidão do TensorFlow persiste no back-end do TFLite.

In [ ]:
# Keras model evaluation
stripped_sparsity_clustered_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
_, sparsity_clustered_keras_accuracy = stripped_sparsity_clustered_model.evaluate(
    test_images, test_labels, verbose=0)

# TFLite model evaluation
interpreter = tf.lite.Interpreter(pruned_and_clustered_tflite_file)
interpreter.allocate_tensors()

sparsity_clustered_tflite_accuracy = eval_model(interpreter)

print('Pruned, clustered and quantized Keras model accuracy:', sparsity_clustered_keras_accuracy)
print('Pruned, clustered and quantized TFLite model accuracy:', sparsity_clustered_tflite_accuracy)

## Conclusão

Neste tutorial, você aprendeu a criar um modelo, fazer o pruning dele usando a API `prune_low_magnitude()` e aplicar o clustering que preserva a esparsidade ao agrupar os pesos. O modelo que passou pelo clustering que preserva a esparsidade foi comparado a um modelo agrupado para mostrar que a esparsidade é preservada no primeiro e perdida no último. Em seguida, o modelo agrupado e podado foi convertido para o TFLite, mostrando os benefícios da compressão ao usar as técnicas de pruning e clustering que preserva a esparsidade em cadeia para a otimização do modelo. Por fim, o modelo do TFLite foi avaliado para garantir a persistência da exatidão no back-end do TFLite.